# Cleanup from Facebook post.json

In [23]:
import json
import pandas as pd

In [24]:
file = "raw/posts/posts.json"

In [25]:
# read from file
with open(file) as json_data:
    data = json.load(json_data)
su = data['status_updates']

In [26]:
counter = 0
for item in su:
    if counter > 1000:
        break
    
    counter += 1
    
    print(item)
    print(item['timestamp']) 
    print(len(item['data'])) 
    if len(item['data']) == 1:
        print(item['data'])

{'timestamp': 1553080668, 'data': [{'post': 'Wahlkampf ist dieser Tage Ã¼berall in der TÃ¼rkei. Gleich lang sind die Spiesse im Kampf um die Aufmerksamkeit der WÃ¤hler allerdings nicht â\x80\x93 am wenigsten in den Medien. \n\nEin Auftritt des PrÃ¤sidenten ist stets Â«Breaking NewsÂ». ð\x9f\x97\x9e'}, {'update_timestamp': 1553080668}], 'title': 'NZZ Neue ZÃ¼rcher Zeitung shared a link.'}
1553080668
2
{'timestamp': 1553080150, 'data': [{'post': 'Ein Gerichtsentscheid gegen den Monsanto-Besitzer Bayer weist weit Ã¼ber den Einzelfall hinaus: Er zeigt, dass selbst bei guter Evidenz fÃ¼r die eigene Sache einer Firma Milliardenzahlungen drohen kÃ¶nnen. \n\nBerechenbarkeit ist keine StÃ¤rke des Wirtschaftsstandortes USA. Unser Kommentar:'}, {'update_timestamp': 1553080150}], 'title': 'NZZ Neue ZÃ¼rcher Zeitung shared a link.'}
1553080150
2
{'timestamp': 1553074598, 'data': [{'post': 'Mal eben schnell von ZÃ¼rich fÃ¼r 65 Franken nach Mallorca oder von Basel fÃ¼r 77 Franken nach Berlin oder von

1
[{'post': 'Kim Jong Un ist in Hanoi angekommen und warm empfangen worden. In der nordvietnamesischen Provinz treffen sich US-PrÃ¤sident Donald J. Trump und der nordkoreanische FÃ¼hrer zum zweiten Mal. \n\nDie Hanoianer freuen sich, dass ein so bedeutendes Ereignis bei ihnen stattfindet. Und machen krÃ¤ftig Profit. Die Highlights im Video.'}]
{'timestamp': 1551179056, 'data': [{'post': 'Der Politikwissenschafter Ian Bremmer erklÃ¤rt, wie wir uns im Zeitalter des Populismus ideell abschotten â\x80\x93 und was wir dagegen tun kÃ¶nnen.\n\nMarc Neumann hat ihn zum Interview getroffen:'}, {'update_timestamp': 1551179056}], 'title': 'NZZ Neue ZÃ¼rcher Zeitung shared a link.'}
1551179056
2
{'timestamp': 1551177494, 'data': [{'post': 'ð\x9f\x92¬ Â«Wir brauchen einen Climate-Change, den aber gibt es nur mit einem System-ChangeÂ» \nð\x9f\x92¬ Â«Ich bin 18 und kÃ¶nnte einfach chillen, aber nein, ich muss mir Gedanken darÃ¼ber machen, wie wir es besser machen kÃ¶nntenÂ»\n\nDer Sinn des Lebens spi

In [27]:
counter = 0
rows = []

posts_without_comments = 0
posts_with_comments = 0

for item in su:
    if counter > 50000:
        break
    counter += 1
    
    att = item.get('attachments', False)
    media = att[0]['data'][0].get('media', False) if att else False
    
    # check if comments in this post
    if(media and media.get('comments', False)):
        
        for comment in media.get('comments'):
            
            # comment infos                
            row = {}
            
            row['comment__text'] = comment.get('comment').encode('latin1').decode('utf8')
            row['comment__author'] = comment.get('author').encode('latin1').decode('utf8')
            row['comment__timestamp'] = comment.get('timestamp')
    
            # context info - in this case the facebook-post 
            row['post__timestamp'] = item.get('timestamp', '')
            row['post__title'] = item.get('title', '').encode('latin1').decode('utf8')
            row['post__text'] = item.get('data')[0].get('post', '').encode('latin1').decode('utf8') if item.get('data', False) else ''

            rows.append(row)
            
        posts_with_comments += 1
    else: 
        posts_without_comments += 1

print("Posts without comments: {} / Posts with comments: {} / Comments: {}".format(posts_without_comments, posts_with_comments, len(rows)))

Posts without comments: 42882 / Posts with comments: 2482 / Comments: 60599


In [28]:
from datetime import datetime

# Load into Dataframe
df = pd.DataFrame(rows)

# Add human readebale datetime
date_format = '%d.%m.%Y %H:%M:%S'

df['comment__datetime'] = df['comment__timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime(date_format))
df['post__datetime'] = df['post__timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime(date_format))

df

,comment__author,comment__text,comment__timestamp,post__text,post__timestamp,post__title,comment__datetime,post__datetime
0,Sandro Jäger,"Michael wie du, aber er wird bezahlt dafür",1552910440,Er hat sich den Schoggijob ergattert: Maître C...,1552905762,,18.03.2019 13:00:40,18.03.2019 11:42:42
1,Dawid Mazur,ich weiss nicht ob das so gut ist??,1552813681,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:08:01,14.03.2019 17:32:43
2,Lutz Ackermann,Never ever Smart Home!,1552813943,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:12:23,14.03.2019 17:32:43
3,Alexander Eickhoff,Ist es wirklich Fortschritt wenn der Mensch im...,1552814031,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:13:51,14.03.2019 17:32:43
4,Katrin Senk,Und der Spiegel schickt sofort ein Storno des ...,1552814043,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:14:03,14.03.2019 17:32:43
5,Harry Rosenbladt,"Der Kühlschrank wird mit dem Klo vernetzt, der...",1552814307,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:18:27,14.03.2019 17:32:43
6,Ute Knöpel,Wer will denn sowas¿?,1552814316,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:18:36,14.03.2019 17:32:43
7,Bunjamin Caili,Never!,1552814664,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:24:24,14.03.2019 17:32:43
8,Frank Nadler,Die Verbrecher durchstreifen schon die Straßen...,1552814681,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:24:41,14.03.2019 17:32:43
9,Frank Nadler,Ich sage immer: wir sind auf dem besten Weg zu...,1552814782,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:26:22,14.03.2019 17:32:43


In [29]:
# hash user names
from hashlib import md5
df['comment__author'] = df['comment__author'].apply(lambda x: md5(x.encode('utf8')).hexdigest())

df.head(10)

,comment__author,comment__text,comment__timestamp,post__text,post__timestamp,post__title,comment__datetime,post__datetime
0,ae0d65d461f6a8b60bf7e670c48e8c27,"Michael wie du, aber er wird bezahlt dafür",1552910440,Er hat sich den Schoggijob ergattert: Maître C...,1552905762,,18.03.2019 13:00:40,18.03.2019 11:42:42
1,5ef5871b19671c6658f87c0b2372badd,ich weiss nicht ob das so gut ist??,1552813681,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:08:01,14.03.2019 17:32:43
2,73657a199599e86d08584fc0b159a24b,Never ever Smart Home!,1552813943,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:12:23,14.03.2019 17:32:43
3,2ba0de7fc781ebd126a952fbb194a879,Ist es wirklich Fortschritt wenn der Mensch im...,1552814031,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:13:51,14.03.2019 17:32:43
4,9823a75df5bb679a7c5a179649a5eac2,Und der Spiegel schickt sofort ein Storno des ...,1552814043,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:14:03,14.03.2019 17:32:43
5,681507500caadfc25967436869cb99b7,"Der Kühlschrank wird mit dem Klo vernetzt, der...",1552814307,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:18:27,14.03.2019 17:32:43
6,f3927a4edd52540f29db59ddeccb9440,Wer will denn sowas¿?,1552814316,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:18:36,14.03.2019 17:32:43
7,850cf03190b933ffe8a29d7dbd791526,Never!,1552814664,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:24:24,14.03.2019 17:32:43
8,c43bd5c261596f4e005f3660460df435,Die Verbrecher durchstreifen schon die Straßen...,1552814681,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:24:41,14.03.2019 17:32:43
9,c43bd5c261596f4e005f3660460df435,Ich sage immer: wir sind auf dem besten Weg zu...,1552814782,Der Kühlschrank bestellt unser Lieblingsessen ...,1552581163,,17.03.2019 10:26:22,14.03.2019 17:32:43


In [30]:
# write current state to csv
df.to_csv("nzz_facebook_kommentare.csv", sep=';', encoding='utf8')